In [ ]:
import os
import random
import sys
from genetic_algo import Formula, GeneticAlgorithm
from evolution_methods import *

: 

In [ ]:
# import visualization libraries
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


: 

In [ ]:
import math

steepness_param = 100
midpoint_curve = 0.8
# plot logistic function
# success_ratio = 1 / (1 + math.exp(-steepness_param * (x - midpoint_curve)))

def f(x):
    return 1 / (1 + math.exp(-steepness_param * (x - midpoint_curve)))

datay = [ f(x) for x in np.arange(0, 1, 0.01)]
datax = [ x for x in np.arange(0, 1, 0.01)]

plt.ion()
plt.plot(datax, datay)
# make a vertical line in the plot
plt.axvline(x=midpoint_curve, color='r', linestyle='--')
plt.show()


: 

In [ ]:
# load data from results.out
def get_loaded_data(file_name):
    results = {}
    with open(file_name, "r") as f:
        current_file = ""
        iteration_count = -1
        for line in f:

            if "Solving file:" in line:
                current_file = line.split(" ")[-1].strip()
                results[current_file] = {"solution": [], "iterations": []}
                continue

            if "Iteration" in line:
                results[current_file]["iterations"].append({"fitness": 0, "success_rate": 0.0})
                iteration_count += 1
                continue

            if "The best fitness" in line:
                results[current_file]["iterations"][iteration_count]["fitness"] = float(line.split(":")[-1])
                continue

            if "configuration success rate" in line:
                results[current_file]["iterations"][iteration_count]["success_rate"] = float(line.split(":")[-1].strip())
                continue

            if "Solution" in line:
                results[current_file]["solution"] = line.split(":")[-1]
                iteration_count = -1
                continue
    return results

: 

In [ ]:
# generate graphs where all our fitness values and success_rates are lines in graph

results = get_loaded_data("results/boosted_example.out")
for file in list(results.keys())[:4]:
    fitness_line = []
    success_rate_line = []

    for result in results[file]["iterations"]:
        fitness_line.append(result["fitness"])
    # plt.ion()
    plt.plot(fitness_line)

plt.ylabel("Fitness value")
plt.xlabel("Number of iterations")
# plt.legend(list(results.keys())[0:20])
plt.show()

: 

# 'Dobre' napady na "experimenty"

- jedno nastavenie spustime viackrat, aby sme porovnali rozsah nahodnosti
- porovnanie kazdej metody (selekcia, krizenie, fitness, atd.) nad jednym reprezentativnym suborom. Kazda metoda sa spusti niekolkokrat a vypocitame priemer. Vysledky dame do jedneho grafu
- porovnanie 3-4 roznych nastaveni genetickeho algoritmu (kombinacia roznych metod dokopy) nad viacerymi (5-10) subormi. Kazde sa spusti viackrat a vytvorime tak niekolko (5-10) grafov, v kazdom porovname priemerne vysledky daneho nastavenia.
- program spustime nad (asi) vsetkymi instanciami a porovname, ci nasiel pre vsetky riesenia


In [ ]:
# define one specific build
def build1():
    return GeneticAlgorithm(population_size=50, reproduction_count=30, new_blood=20, elitism=True, survivors=5, max_iterations=100) \
        .set_initial_population_method(RandomInitialPopulation()) \
        .set_fitness_function(SuccessRateFitnessFunction(strict_satisfiability=True)) \
        .set_selection_method(RouletteSelection()) \
        .set_crossover_method(SinglePointCrossover()) \
        .set_mutation_method(BitFlipMutation(mutation_chance=0.01))

: 

In [ ]:
# define one specific build
def build2():
    return GeneticAlgorithm(population_size=50, reproduction_count=30, new_blood=20, elitism=True, survivors=5, max_iterations=100) \
        .set_initial_population_method(RandomInitialPopulation()) \
        .set_fitness_function(SuccessRateFitnessFunction(strict_satisfiability=False)) \
        .set_selection_method(RouletteSelection()) \
        .set_crossover_method(SinglePointCrossover()) \
        .set_mutation_method(BitFlipMutation(mutation_chance=0.01))

: 

In [ ]:
# define one specific build
def build3():
    return GeneticAlgorithm(population_size=50, reproduction_count=30, new_blood=20, elitism=True, survivors=5, max_iterations=100) \
        .set_initial_population_method(RandomInitialPopulation()) \
        .set_fitness_function(PunishedSuccessRateFitnessFunction(strict_satisfiability=False)) \
        .set_selection_method(RouletteSelection()) \
        .set_crossover_method(SinglePointCrossover()) \
        .set_mutation_method(BitFlipMutation(mutation_chance=0.01))

: 

In [ ]:
# define one specific build
def build4():
    return GeneticAlgorithm(population_size=100, reproduction_count=50, new_blood=70, elitism=False, survivors=10, max_iterations=100) \
        .set_initial_population_method(RandomInitialPopulation()) \
        .set_fitness_function(SuccessRateFitnessFunction(strict_satisfiability=True)) \
        .set_selection_method(RouletteSelection()) \
        .set_crossover_method(SinglePointCrossover()) \
        .set_mutation_method(BitFlipMutation(mutation_chance=0.01))

: 

In [ ]:
def find_solution(filename, build):
    formula = Formula(filename)
    # create evolution algorithm
    evolution_algorithm = build()
    solution = evolution_algorithm.solve(formula)

    print("Solution:", solution)
    print("Weight:", formula.get_total_weight(solution))

: 

In [ ]:
filename = "data/wuf20-71-M/wuf20-01.mwcnf"
find_solution(filename, build1)

: 

In [ ]:
filename = "data/wuf20-71-M/wuf20-01.mwcnf"
find_solution(filename, build2)

: 

In [ ]:
filename = "data/wuf20-71-M/wuf20-01.mwcnf"
find_solution(filename, build3)

: 

In [ ]:
filename = "data/wuf100-430-Q/wuf100-01.mwcnf"
find_solution(filename, build1)

: 

In [ ]:
# this will calculate if the solution is optimal
from test import OptimalityTester
tester = OptimalityTester()

In [ ]:
filename = "data/wuf20-71-M/wuf20-01.mwcnf"
tester.find_solution(filename)